# Specification of the Genetic algorithm
## Parameters: 
- the dimension of fitness should be population size 
- there are five inequality constrains
- mutation probability: mp (0.59)
- crossover probability: cp (0.15)
- number of parents at each iteration: num_parents (tested with 6)

In [69]:
import numpy as np
from IPython.core.debugger import set_trace
def cal_pop_fitness(pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = [-(1060-pop_i[0]**2-4*pop_i[1]**2-pop_i[2]**2-pop_i[0]*pop_i[1]-pop_i[0]*pop_i[2]) for pop_i in pop]
    return np.array(fitness) #

def cal_pop_cv(pop):
    epislon = 10
    nv = np.zeros(pop.shape[0])
    cv = np.zeros(pop.shape[0])
    for i in range(pop.shape[0]):
        for j in range(pop.shape[1]):
            if pop[i][j]<0:
                nv[i]+=1
                cv[i] += np.abs(pop[i][j])
        if np.abs(np.sum(pop[i]**2)-35)>epislon:
            nv[i]+=1
            cv[i]+=np.abs(np.sum(pop[i]**2)-35)
        if np.abs(np.dot(np.array([10,13,7]),pop[i])-65)>epislon:
            nv[i]+=1
            cv[i]+=np.abs(np.dot(np.array([10,13,7]),pop[i])-65)
    return nv,cv

def select_mating_pool(pop, fitness, nv,cv,num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    # seperate the pop into constrains satisfied group and unsatsfied group
    pop_stsfd_idx = np.where(nv==0)[0]
    pop_unstsfd_idx = np.where(nv!=0)[0]
    parents = np.empty((num_parents, pop.shape[1]))
#     set_trace()
    if num_parents <=len(pop_stsfd_idx): # we have enough constrains satisfied candidates
        for parent_num in range(num_parents):
            max_fitness_idx = np.where(fitness == np.max(fitness))
            max_fitness_idx = max_fitness_idx[0][0]
            parents[parent_num, :] = pop[max_fitness_idx, :]
            fitness[max_fitness_idx] = -99999999999
    else:
        nv_threshold = 1
        while num_parents > len(np.where(nv<=nv_threshold)[0]):
            nv_threshold+=1
        if len(np.where(nv<=nv_threshold-1)[0])!=0:
            for i in range(len(np.where(nv<=nv_threshold-1)[0])):
                min_nv_idx = np.where(nv == np.min(nv))
                min_nv_idx = min_nv_idx[0][0]
                parents[i, :] = pop[min_nv_idx, :]
                nv[min_nv_idx] = 99
        num_parents_needed = num_parents - len(np.where(nv<=nv_threshold-1)[0])
        nv_threshold_idx = np.where(nv == nv_threshold)[0]
        pop_nv_equals_threshold = pop[nv_threshold_idx]
        fitness_nv_equals_threshold = fitness[nv_threshold_idx]
        for parent_needed in range(num_parents_needed):
            max_fitness_idx = np.where(fitness_nv_equals_threshold == np.max(fitness_nv_equals_threshold))
            max_fitness_idx = max_fitness_idx[0][0]
            parents[len(np.where(nv<=nv_threshold-1)[0])+parent_needed, :] = pop_nv_equals_threshold[max_fitness_idx, :]
            fitness_nv_equals_threshold [max_fitness_idx] = -99999999999
    return parents



def crossover(parents, offspring_size,cp):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        cs_succeed = np.random.uniform(low=0, high=1)
        if cs_succeed >= cp:
            # Index of the first parent to mate.
            parent1_idx = k%parents.shape[0]
            # Index of the second parent to mate.
            parent2_idx = (k+1)%parents.shape[0]
            # The new offspring will have its first half of its genes taken from the first parent.
            offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
            # The new offspring will have its second half of its genes taken from the second parent.
            offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
        else:
            parent1_idx = k%parents.shape[0]
            offspring[k,:] = parents[parent1_idx]
    return offspring

def mutation(pop,mp):
    # Mutation changes a single gene in each individual of the current population randomly.
    # mp stands for mutation probability
    for idx in range(pop.shape[0]):
        for var_idx in range(pop.shape[1]):
            mt_succeed = np.random.uniform(low=0, high=1)
            if mt_succeed >= mp: 
                # The random value to be added to the gene.
                random_value = np.random.uniform(-0.5, 0.5, 1)
                pop[idx, var_idx] = pop[idx, var_idx] + random_value
    return pop


# Define the GA process

In [168]:
def ga(pop, num_generations,num_parents):
    import numpy as np
    pop_history =[]
    fitness_history = []
    nv_history = []
    cv_history =[]
    for generation in range(num_generations):
        fitness = cal_pop_fitness(pop)
        nv,cv = cal_pop_cv(pop)
        parents=select_mating_pool(pop,fitness,nv,cv,num_parents)
        offspring_crossover = crossover(parents,offspring_size=(pop_size-parents.shape[0], num_var),cp = 0.59)
        offspring_crossover = mutation(offspring_crossover,mp =0.15)
        pop = np.empty(pop.shape)
        pop[0:parents.shape[0], :] = parents
        pop[parents.shape[0]:, :] = offspring_crossover
#         pop = mutation(pop,mp =0.15)
        pop_history.append(pop)
        fitness = -cal_pop_fitness(pop)
        best_idx = np.where(fitness == np.max(fitness))
        fitness_history.append(fitness[best_idx[0][0]])
        nv_history.append(nv[best_idx[0][0]])
        cv_history.append(cv[best_idx[0][0]])
    fitness = -cal_pop_fitness(pop)
    best_idx = np.where(fitness == np.max(fitness))
    return pop[best_idx[0][0]],fitness[best_idx[0][0]],pop_history,fitness_history,nv_history,cv_history



# main function and animation

In [171]:
import numpy as np
pop_size = 40
num_var = 3 # x1,x2, x3
init_pop = np.random.uniform(low=0, high=4, size=(pop_size,num_var))
best_var, objective, pop_history, fitness_history,nv_history,cv_history = ga(init_pop,800,6)
print(objective)

903.7001175472127


In [173]:
import matplotlib.pyplot as plt
x = range(len(fitness_history))
# plt.scatter(x,nv_history,  color='blue',linewidths=0.01)
# plt.title('N.V. tendency')
# plt.xlabel('iterations')
# plt.ylabel('number of constrain violations')
# fig= plt.gcf()
# fig.set_size_inches(12,4)

plt.plot(x,cv_history,  color='blue',linewidth=0.5)
plt.title('C.V. tendency')
plt.xlabel('iterations')
plt.ylabel('constrain violations')
fig= plt.gcf()
fig.set_size_inches(12,4)

# plt.plot(x,nv_history, marker='', color='olive', linewidth=2)
# plt.plot( x,cv_history, marker='', color='olive', linewidth=2, linestyle='dashed', label="toto")
# fig.legend()


In [164]:
# animation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import seaborn as sns

def animate_scatters(iteration, data, scatters):
    """
    Update the data held by the scatter plot and therefore animates it.
    Args:
        iteration (int): Current iteration of the animation
        data (list): List of the data positions at each iteration.
        scatters (list): List of all the scatters (One per element)
    Returns:
        list: List of scatters (One per element) with new coordinates
    """
    for i in range(data[0].shape[0]):
        scatters[i]._offsets3d = (data[iteration][i,0:1], data[iteration][i,1:2], data[iteration][i,2:])
    return scatters
def constrain1 (x,y):
    return np.sqrt(35-x**2-y**2)
def constrain2 (x,y):
    return -np.sqrt(35-x**2-y**2)

fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('x3')
ax.set_xlim3d([-4, 4])
ax.set_ylim3d([-4, 4])
ax.set_zlim3d([-4, 4])
x =y= np.linspace(-4.2,4.2)
X, Y = np.meshgrid(x, y)
zs = np.array(constrain1(np.ravel(X), np.ravel(Y)))
Z = zs.reshape(X.shape)
ax.plot_surface(X, Y, Z)
# next sphere
x =y= np.linspace(-4.2,4.2)
X, Y = np.meshgrid(x, y)
zs = np.array(constrain2(np.ravel(X), np.ravel(Y)))
Z = zs.reshape(X.shape)
ax.plot_surface(X, Y, Z)
ax.set_title('Population evolution',fontsize=20)
scatters = [ax.scatter(pop_history[0][i,0],pop_history[0][i,1],pop_history[0][i,2], c='skyblue',s=60) for i in range(pop_history[0].shape[0])]
iterations = len(pop_history)
ni = animation.FuncAnimation(fig, animate_scatters, iterations, fargs=(pop_history, scatters),
                                       interval=10, blit=False, repeat=True)
plt.show()

F:\software\2PYTHON\envs\deepLearning\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in sqrt
F:\software\2PYTHON\envs\deepLearning\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Z contains NaN values. This may result in rendering artifacts.
F:\software\2PYTHON\envs\deepLearning\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in sqrt
F:\software\2PYTHON\envs\deepLearning\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Z contains NaN values. This may result in rendering artifacts.


In [155]:
x = np.linspace(-1,1)
y=np.linspace(-1,1)
z=np.sqrt(35-x**2-y**2)
z

array([5.74456265, 5.75846623, 5.77175908, 5.78444539, 5.79652914,
       5.80801411, 5.81890383, 5.82920165, 5.83891068, 5.84803388,
       5.85657396, 5.86453349, 5.87191481, 5.87872012, 5.88495141,
       5.89061049, 5.89569902, 5.90021848, 5.90417016, 5.90755522,
       5.91037462, 5.91262917, 5.91431952, 5.91544615, 5.91600938,
       5.91600938, 5.91544615, 5.91431952, 5.91262917, 5.91037462,
       5.90755522, 5.90417016, 5.90021848, 5.89569902, 5.89061049,
       5.88495141, 5.87872012, 5.87191481, 5.86453349, 5.85657396,
       5.84803388, 5.83891068, 5.82920165, 5.81890383, 5.80801411,
       5.79652914, 5.78444539, 5.77175908, 5.75846623, 5.74456265])

In [66]:
# Scripts cell
import numpy as np
pop_size = 40
num_var = 3 # x1,x2, x3
init_pop = np.random.uniform(low=0, high=5, size=(pop_size,num_var))
nv,cv = cal_pop_cv(init_pop)
fitness = cal_pop_fitness(init_pop)
parents=select_mating_pool(init_pop,fitness,nv,cv,6)
offspring_crossover = crossover(parents,offspring_size=(pop_size-parents.shape[0], num_var),cp = 0.59)
pop = np.empty((pop_size,num_var))
pop[0:parents.shape[0], :] = parents
pop[parents.shape[0]:, :] = offspring_crossover
mutation(pop,mp =0.15)
# cal_pop_fitness(pop)

array([[ 4.44574992,  4.45288134,  1.83606088],
       [ 1.41518531,  4.94523046,  4.9625359 ],
       [ 2.92276199,  4.1782813 ,  3.76001318],
       [ 0.15202501,  4.81626271,  4.90875922],
       [ 2.76744925,  4.6906161 ,  2.03764093],
       [ 4.94969121,  4.17883073, -0.33239069],
       [ 4.80089382,  4.85023526,  1.1214722 ],
       [ 1.4448909 ,  5.22778234,  5.23864391],
       [ 3.74266676,  4.64369814,  3.95445414],
       [-0.17009137,  4.84084356,  4.27952483],
       [ 2.01685722,  3.69667677,  0.19297276],
       [ 4.2224136 ,  4.53929843,  1.75586353],
       [ 4.68316492,  4.67496185,  1.41873379],
       [ 0.99484798,  4.78042458,  4.48121127],
       [ 3.54600128,  4.16409903,  3.73089299],
       [ 0.25135862,  5.22155231,  2.56946487],
       [ 2.10514049,  4.40932123,  2.19379807],
       [ 4.51034573,  3.9265677 , -0.14306027],
       [ 5.12687259,  5.08819718,  4.63715109],
       [ 0.89026651,  4.78042458,  5.12572978],
       [ 3.39443511,  3.89094536,  3.758